In [ ]:
import boto3 

my_session = boto3.session.Session()
my_region = my_session.region_name
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
algorithm_name = "facenet"

In [ ]:
#remove ipython from requirement.txt
!./build_and_push.sh

In [ ]:
!cat Dockerfile

In [ ]:
!cat build_and_push.sh

In [ ]:
image_uri=f"{account_id}.dkr.ecr.{my_region}.amazonaws.com/{algorithm_name}:latest"
image_uri

In [ ]:
import sagemaker 
sess = sagemaker.session.Session()
bucket = sess.default_bucket()

In [ ]:
import sagemaker 
session = sagemaker.session.Session()
bucket = session.default_bucket()

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

s3_path = f"s3://{bucket}/data/faces"
s3_path

In [ ]:
# s3://facenet-demo-yianc 
# input.zip
!mkdir input 
!mkdir input/data
!mv examples/lfw input/data/faces


!aws s3 cp --recursive ./input/data/faces/ $s3_path

### can train in docker container manually 

* start container in interactive mode
```
docker run -it -v $PWD:/opt/ml --entrypoint '' $image_uri bash 
docker run -it -v $PWD:/opt/ml $image_uri bash 
```

In [ ]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "facenet-{}".format(str(int(timestamp))) 
job_name

In [ ]:
estimator = sagemaker.estimator.Estimator( 
                        role=role,
                        image_uri=image_uri,
                        instance_count=1,
                        instance_type='ml.p3.8xlarge',
                        sagemaker_session=session,
                        volume_size=100, 
                        debugger_hook_config=False
                   )

In [ ]:
estimator.fit(inputs={"faces":s3_path}, job_name=job_name)

In [ ]:
model_path = estimator.model_data

In [ ]:
!aws s3 cp $model_path . 
!tar -xf model.tar.gz
!ls -al fine_tuned_model
